In [88]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.svm import LinearSVR, SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

## Use this DF from here on with the filled Price

In [89]:
df = pd.read_csv(
    './data/Modelar_UH2021_filled_precio.txt', parse_dates=[1], index_col=0
)

/home/muriel/.local/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [90]:
df_est = pd.read_csv(
    './data/Estimar_UH2021_filled_precio.txt', parse_dates=[1], index_col=0
)

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Feature engineering

### Modelar

In [91]:
df=df.drop_duplicates()

conditions = [ (df["estado"] == 'No Rotura'), (df["estado"] == 'Transito'), (df["estado"] == 'Rotura') ]
values = [1, 0, -1]
df["estado_num"] = np.select(conditions, values)

df["weekday"] = df["fecha"].dt.weekday
df["antiguedad"] = df["antiguedad"].astype('Int64')
df["antiguedad_std"] = df["antiguedad"]-df["antiguedad"].min()
df["categoria_dos"] = df["categoria_dos"].astype("Int64")

df = df.drop(columns=["estado", "antiguedad"])

In [92]:
# Cyclical features
df['weekday_sin'] = np.sin(df.fecha.dt.weekday * (2*np.pi/7))
df['weekday_cos'] = np.cos(df.fecha.dt.weekday * (2*np.pi/7))

month_con = df["fecha"].dt.month + (df["fecha"].dt.day / df["fecha"].dt.days_in_month)
df['month_sin'] = np.sin((month_con-1) * (2*np.pi/12))
df['month_cos'] = np.cos((month_con-1) * (2*np.pi/12))

In [ ]:
# AQUÍ
# One-hot encoding of categoria_dos
df["categoria_dos"].unique()
df['categoria_dos_'+str()] = (df.dia_atipico == -1).astype(int)
df['categoria_dos_'+str] = (df.dia_atipico == 0).astype(int)
df['dia_atipico_pos'] = (df.dia_atipico == 1).astype(int)

### Estimar

In [93]:
df_est=df_est.drop_duplicates()

conditions = [ (df_est["estado"] == 'No Rotura'), (df_est["estado"] == 'Transito') ]
values = [1, 0]
df_est["estado_num"] = np.select(conditions, values)

df_est["weekday"] = df_est["fecha"].dt.weekday
df_est["antiguedad"] = pd.to_numeric(df_est["antiguedad"], errors='coerce') 
df_est["antiguedad"] = df_est["antiguedad"].astype('Int64')
df_est["antiguedad_std"] = df_est["antiguedad"]-df_est["antiguedad"].min()
df_est["categoria_dos"] = pd.to_numeric(df_est["categoria_dos"], errors='coerce') 
df_est["categoria_dos"] = df_est["categoria_dos"].astype("Int64")

df_est = df_est.drop(columns=["estado", "antiguedad"])

In [94]:
# Cyclical features
df_est['weekday_sin'] = np.sin(df_est.fecha.dt.weekday * (2*np.pi/7))
df_est['weekday_cos'] = np.cos(df_est.fecha.dt.weekday * (2*np.pi/7))

month_con = df_est["fecha"].dt.month + (df_est["fecha"].dt.day / df_est["fecha"].dt.days_in_month)
df_est['month_sin'] = np.sin((month_con-1) * (2*np.pi/12))
df_est['month_cos'] = np.cos((month_con-1) * (2*np.pi/12))

### Further tuning

In [95]:
# Multiply x5 number of visits before 25/1/2021
conditions = [ (df["fecha"] >= datetime.datetime(2016,1,25)), (df["fecha"] < datetime.datetime(2016,1,25)) ]
values = [df["visitas"], df["visitas"]*5]
df["visitas"] = np.select(conditions, values)

In [31]:
# (NO MILLORA) Add feature visits 1 day before 
# It only works if there are no missing entries and they are sorted (!!)
df = df.sort_values(by = ['categoria_uno','id', 'fecha']).reset_index(drop=True)

# Create transitional previous day visits feature
first_element = pd.Series([0])
visits_day_before = first_element.append(df["visitas"], ignore_index=True)
df["visitas_dia_antes"] = visits_day_before

# Select all the first entries from the stations and set the previous entries and the tendencies to 0
product_unique_ids = sorted(df["id"].unique())

for product_id in product_unique_ids:
    first_id_index = df[df["id"] == product_id].index[0]
    df.at[first_id_index, "visitas_dia_antes"] = 0        

In [32]:
# (NO MILLORA) Add feature visits 7 days before 
# It only works if there are no missing entries and they are sorted (!!)
df = df.sort_values(by = ['categoria_uno','id', 'fecha']).reset_index(drop=True)

# Create transitional 7-day previous visits feature
first_element = pd.Series([0, 0, 0, 0, 0, 0, 0])
visits_7_days_before = first_element.append(df["visitas"], ignore_index=True)
df["visitas_7_dias_antes"] = visits_7_days_before

# Select all the first entries from the stations and set the previous entries and the tendencies to 0
product_unique_ids = sorted(df["id"].unique())

for product_id in product_unique_ids:
    first_id_index = df[df["id"] == product_id].index[0]
    for extra_day in range(7):
        df.at[first_id_index+extra_day, "visitas_7_dias_antes"] = 0        

In [122]:
# Only use data before the pattern change
#df = df[df.fecha < datetime.datetime(2016,1,24)]
#df_est = df_est[df_est.fecha < datetime.datetime(2016,1,24)]
# Only use data after the pattern change
#df = df[df.fecha > datetime.datetime(2016,1,25)]
#df_est = df_est[df_est.fecha > datetime.datetime(2016,1,25)]

In [122]:
# Only use data from typical days outside of campaigns
df = df.loc[(df["dia_atipico"] == 0) & (df["campaña"] == 0), ]
df_est = df_est.loc[(df_est["dia_atipico"] == 0) & (df_est["campaña"] == 0), ]

In [97]:
# Drop unwanted columns
df = df.drop(columns=["fecha", "id", "weekday"])
df_est = df_est.drop(columns=["fecha", "id", "weekday"])
#df = df.drop(columns=["fecha", "id", "weekday_sin", "weekday_cos"])
#df_est = df_est.drop(columns=["fecha", "id", "weekday_sin", "weekday_cos"])

In [98]:
# Drop nans
df = df.dropna()
df_est = df_est.dropna()

### Split dataset in categories

In [99]:
# Split dataset in categorias_uno and limit number of training samples per categoria_uno
number_samples_desired = 1000000

list_categoria_uno = sorted( df["categoria_uno"].unique() )
data_cat = [None]*len(list_categoria_uno)
data_est_cat = [None]*len(list_categoria_uno)

for index in range(len(list_categoria_uno)):

    number_samples = number_samples_desired
    number_samples_available = len(df[df["categoria_uno"] == list_categoria_uno[index]])
    if number_samples_desired > number_samples_available:
        number_samples = number_samples_available
        print(f"Only {number_samples_available} samples for categoria_uno = {list_categoria_uno[index]}")
        
    # Modelar
    data_cat[index] = df[df["categoria_uno"] == list_categoria_uno[index]].sample(n=number_samples, random_state=0)
    data_cat[index] = data_cat[index].drop(columns = "categoria_uno")
    # Estimar
    data_est_cat[index] = df_est[df_est["categoria_uno"] == list_categoria_uno[index]]
    data_est_cat[index] = data_est_cat[index].drop(columns = "categoria_uno")

Only 492895 samples for categoria_uno = A
Only 43044 samples for categoria_uno = B
Only 137572 samples for categoria_uno = C
Only 422 samples for categoria_uno = D
Only 119848 samples for categoria_uno = E
Only 98746 samples for categoria_uno = F
Only 70896 samples for categoria_uno = G
Only 112674 samples for categoria_uno = H
Only 26586 samples for categoria_uno = I
Only 195386 samples for categoria_uno = K
Only 38824 samples for categoria_uno = L
Only 3376 samples for categoria_uno = N
Only 2110 samples for categoria_uno = O


## Train / test split

In [100]:
X = [None]*len(list_categoria_uno)
y = [None]*len(list_categoria_uno)
X_train = [None]*len(list_categoria_uno)
X_test = [None]*len(list_categoria_uno)
y_train = [None]*len(list_categoria_uno)
y_test = [None]*len(list_categoria_uno)

for index in range(len(list_categoria_uno)):
    X[index] = data_cat[index][data_cat[index].columns.difference(["unidades_vendidas"])]
    y[index] = data_cat[index]["unidades_vendidas"]

    X_train[index], X_test[index], y_train[index], y_test[index] = train_test_split(
                                                            X[index], y[index], test_size=0.10, random_state=0)

print(X[0].shape, y[0].shape)
print(len(X_train[0]), len(y_train[0]), len(X_test[0]))
print(X_train[0].shape, y_train[0].shape, X_test[0].shape)

(492895, 11) (492895,)
443605 443605 49290
(443605, 11) (443605,) (49290, 11)


## Normalitzation of selected features

In [101]:
# Normalize some features of the train and test datasets with the Standard Scaler/Robust Scaler 
# Select which columns to use with the scaler

selected_columns = [
#    "fecha",
#    "id",
    "antiguedad_std",
#    "campaña",
#    "categoria_uno",
    "categoria_dos",
#    "dia_atipico",
#    "estado_num",
#    "month_cos",
#    "month_sin",
    "precio",
    "visitas",
#    "weekday",
#    "weekday_cos",
#    "weekday_sin",
#    "visitas_dia_antes",
#    "visitas_7_dias_antes",
    ] 

In [102]:
# I train the scaler with ALL training data available
scaler = StandardScaler().fit(df.loc[:,selected_columns])
#scaler = RobustScaler().fit(df.loc[:,selected_columns])

X_train_scaled = [None]*len(list_categoria_uno)
X_train_non_scaled = [None]*len(list_categoria_uno)
X_test_scaled = [None]*len(list_categoria_uno)
X_test_non_scaled = [None]*len(list_categoria_uno)

for index in range(len(list_categoria_uno)):
    X_train_scaled[index] = scaler.transform(X_train[index].loc[:, selected_columns])
    X_train_non_scaled[index] = X_train[index][X_train[index].columns.difference(selected_columns)]
    X_train_scaled[index] = np.concatenate([X_train_non_scaled[index], X_train_scaled[index]], axis=1)

    X_test_scaled[index] = scaler.transform(X_test[index].loc[:, selected_columns])
    X_test_non_scaled[index] = X_test[index][X_test[index].columns.difference(selected_columns)]
    X_test_scaled[index] = np.concatenate([X_test_non_scaled[index], X_test_scaled[index]], axis=1)

## Train models and predict validation samples

In [103]:
print("Start training: ", datetime.datetime.now())

predictor = [None]*len(list_categoria_uno)
y_predicted = [None]*len(list_categoria_uno)

for index in range(len(list_categoria_uno)):
#    predictor[index] = LinearSVR(random_state=0, tol=1e-5, max_iter=10000)
#    predictor[index] = SVR(kernel='rbf', C=2., tol=1e-5, max_iter=100000)
    predictor[index] = RandomForestRegressor(max_depth=4, random_state=0)
#    predictor[index] = RandomForestRegressor(max_depth=4, n_estimators=200, random_state=0)
    print(predictor[index])

    predictor[index].fit(X_train_scaled[index], y_train[index])
    y_predicted[index] = predictor[index].predict(X_test_scaled[index])

print("End training: ", datetime.datetime.now())

Start training:  2021-03-11 15:25:32.131712
RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=4, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)
RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=4, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=0, ver

## Evaluation

In [104]:
def metrica_atmira(y_test, y_predicted):
    rmse = mean_squared_error(y_test, y_predicted, squared=False)
    rrmse = rmse/y_test.mean()
    # Si el valor és negatiu és que hi ha hagut més demanda de la prevista, si el valor és positiu compta com a CF
    diferencia = y_predicted - y_test
    CF = np.sum(diferencia >= 0)/len(y_test)
    metrica_minimitzar = (0.7*rrmse) + (0.3*(1-CF))
    print("rmse = ", rmse)
    print("y_mean = ", y_test.mean())
    print("rrmse = ", rrmse)
    print("CF =", CF)
    return metrica_minimitzar

In [105]:
# Reconstruct joint y_test and y_predicted
y_test_reconst = pd.Series()
y_predicted_reconst = np.array([])
for index in range(len(list_categoria_uno)):
    y_test_reconst = y_test_reconst.append(y_test[index])
    y_predicted_reconst = np.concatenate((y_predicted_reconst,y_predicted[index]))
    print(list_categoria_uno[index], len(y_test[index]), len(y_predicted[index]), 
          len(y_test_reconst), len(y_predicted_reconst))

A 49290 49290 49290 49290
B 4305 4305 53595 53595
C 13758 13758 67353 67353
D 43 43 67396 67396
E 11985 11985 79381 79381
F 9875 9875 89256 89256
G 7090 7090 96346 96346
H 11268 11268 107614 107614
I 2659 2659 110273 110273
K 19539 19539 129812 129812
L 3883 3883 133695 133695
N 338 338 134033 134033
O 211 211 134244 134244


In [106]:
# Això és usant totes les dades i el RobustScaler
metrica_atmira(y_test_reconst, y_predicted_reconst)

rmse =  11.402529733062975
y_mean =  3.535666398498257
rrmse =  3.225001583267612
CF = 0.7617621644170317


2.3289724589622187

In [87]:
# Això és usant totes les dades
metrica_atmira(y_test_reconst, y_predicted_reconst)

rmse =  11.378664536251144
y_mean =  3.535666398498257
rrmse =  3.218251739215026
CF = 0.7617621644170317


2.3242475681254082

In [68]:
# Això és eliminant els dies atípics i campanyes
metrica_atmira(y_test_reconst, y_predicted_reconst)

rmse =  9.663112763713858
y_mean =  3.2140452017754195
rrmse =  3.006526715422674
CF = 0.7698459262282537


2.1736149229273956

In [44]:
# Això és eliminant els dies atípics i posant les variables visites 1 i 7 dies abans
metrica_atmira(y_test_reconst, y_predicted_reconst)

rmse =  11.867271050707602
y_mean =  3.2308722435496833
rrmse =  3.673085828262064
CF = 0.7682094710931422


2.640697238455502

In [22]:
# Això és eliminant els dies atípics
metrica_atmira(y_test_reconst, y_predicted_reconst)

rmse =  9.834973478745413
y_mean =  3.206614205364317
rrmse =  3.0670897241996156
CF = 0.7687833994413764


2.2163277871073177

In [133]:
# Això és afegint les visites d'1 i 7 dies abans
metrica_atmira(y_test_reconst, y_predicted_reconst)

rmse =  13.198687140539754
y_mean =  3.618860510805501
rrmse =  3.647194220702896
CF = 0.7670677799607073


2.6229156205038144

In [111]:
# Això és afegint les visites de 7 dies abans
metrica_atmira(y_test_reconst, y_predicted_reconst)

rmse =  13.255359153995897
y_mean =  3.618860510805501
rrmse =  3.662854402488551
CF = 0.7670923379174852


2.6338703803667403

In [58]:
metrica_atmira(y_test_reconst, y_predicted_reconst)

rmse =  12.35724949242855
y_mean =  3.6659012770137522
rrmse =  3.3708625952128153
CF = 0.7599950884086444


2.4316052901263774

In [39]:
metrica_atmira(y_test_reconst, y_predicted_reconst)

rmse =  13.424364936164661
y_mean =  3.8621944344795778
rrmse =  3.4758387139495643
CF = 0.7556329968055484


2.5063972007230304

In [20]:
metrica_atmira(y_test_reconst, y_predicted_reconst)

rmse =  11.67899295726826
y_mean =  3.566853482786229
rrmse =  3.274312503620215
CF = 0.477982385908727


2.448624036761532